<a href="https://colab.research.google.com/github/avilaJorge/Policy-Value-Iteration/blob/main/PolicyValueIters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=9)
pd.set_option('precision', 3)

In [ ]:
fns = ['prob_a1.txt', 'prob_a2.txt', 'prob_a3.txt', 'prob_a4.txt']
actions = ['←','↑','→','↓']
n_states = 81
n_rows   = 9
gamma    = 0.99

In [ ]:
trns_mx = []
for fn in fns:
    prob_mx = np.zeros((n_states, n_states))
    with open(fn) as f:
        for l in f.readlines():
            col = l.strip().split()
            assert(len(col) == 3)
            prob_mx[int(col[0]) - 1, int(col[1]) - 1] = float(col[2])
        trns_mx.append(prob_mx)

# Check that matrices were loaded correctly
for m in trns_mx:
    for i in range(n_states):
        assert(np.sum(m[i, :]) == 1)

R_s = []
with open('rewards.txt') as f:
    for l in f.readlines():
        R_s.append(int(l))
R_s = np.array(R_s)

## Policy Iteration

In [ ]:
def policy_eval(policy):
    V_pi = np.zeros((n_states, n_states))
    I    = np.identity(n_states)

    for s in range(n_states):
        a = policy[s]
        V_pi[s,:] = np.identity(n_states)[s,:] - gamma * trns_mx[a][s, :]
    
    V_pi = np.linalg.inv(V_pi)
    V_pi = np.dot(V_pi, R_s)
    return V_pi

In [ ]:
def policy_imp(s, policy):

    ps = []
    for a in range(len(actions)):
        ps.append(np.sum(np.multiply(trns_mx[a][s, :], policy_eval(policy))))
    return np.argmax(ps)

In [ ]:
def policy_iteration():
    policy = np.random.randint(len(actions), size=n_states)

    V_new = policy_eval(policy)
    V_old = np.zeros(n_states)
    while (sum(V_old == V_new) != n_states):
        V_old = np.copy(V_new)

        policy_prime = []
        for s in range(n_states):
            policy_prime.append(policy_imp(s, policy))

        V_new = policy_eval(policy_prime)

        policy = policy_prime

    return policy, V_new

In [ ]:
pi_star, V_star = policy_iteration()

In [ ]:
print_arr = np.flipud(np.rot90(V_star.reshape((9, 9))))
print_list = []
for i in range(9):
    print_row = []
    for j in range(9):
        val = print_arr[i, j]
        if val == 0.0:
            print_row.append('Wall')
        elif val < 0:
            print_row.append('Dragon')
        elif val >= 99.99:
            print_row.append('Exit')
        else:
            print_row.append("%0.4f" % val)
    print_list.append(print_row)

df = pd.DataFrame(print_arr)
df

,0,1,2,3,4,5,6,7,8
0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.00,65.773,67.136,77.846,79.845,72.475,-100.000,0.000,100.000
2,0.00,55.883,-100.000,70.308,81.344,83.048,84.881,96.872,98.719
3,0.00,54.923,50.477,59.666,0.000,80.958,0.000,97.045,98.727
4,53.51,54.146,0.000,-100.000,-100.000,61.780,-100.000,88.220,100.000
5,0.00,52.504,43.936,51.091,61.007,71.786,73.947,85.185,97.573
6,0.00,43.773,-100.000,0.000,0.000,70.351,0.000,-100.000,88.406
7,0.00,47.953,48.769,58.147,59.390,60.169,-100.000,0.000,100.000
8,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [ ]:
def color_map(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val == 'Dragon':
        color = 'red'
    elif val == 'Exit':
        color = 'blue'
    elif val == 'Wall':
        color = 'gray'
    else:
        color = 'black'
    return 'color: %s' % color

In [ ]:
df = pd.DataFrame(print_list)
s = df.style.applymap(color_map)
s

,0,1,2,3,4,5,6,7,8
0,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall
1,Wall,65.7731,67.1365,77.8460,79.8445,72.4751,Dragon,Wall,Exit
2,Wall,55.8829,Dragon,70.3082,81.3444,83.0485,84.8805,96.8723,98.7188
3,Wall,54.9230,50.4766,59.6664,Wall,80.9583,Wall,97.0448,98.7273
4,53.5097,54.1456,Wall,Dragon,Dragon,61.7798,Dragon,88.2204,Exit
5,Wall,52.5040,43.9360,51.0914,61.0072,71.7864,73.9466,85.1846,97.5726
6,Wall,43.7725,Dragon,Wall,Wall,70.3514,Wall,Dragon,88.4059
7,Wall,47.9530,48.7687,58.1474,59.3900,60.1689,Dragon,Wall,Exit
8,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall


In [ ]:
opt_policy_map = []
unexprbl = ['Wall', 'Dragon', 'Exit']
for i in range(9):
    row_list = []
    for j in range(9):
        val = V_star[i * j]
        row_list.append(actions[pi_star[int(i * j)]])
    opt_policy_map.append(row_list)

In [ ]:
arrow_map = np.flipud(np.rot90(np.array(opt_policy_map).reshape((9, 9))))

In [ ]:
opt_policy_map = []
arrow_map = np.flipud(np.rot90(np.array(pi_star).reshape(9, 9)))
for i in range(9):
    row_list = []
    for j in range(9):
        val = print_list[i][j]
        if val in unexprbl:
            row_list.append(val)
        else:
            row_list.append(actions[arrow_map[i, j]])
    opt_policy_map.append(row_list)

In [ ]:
df = pd.DataFrame(opt_policy_map)
s = df.style.applymap(color_map)
s

,0,1,2,3,4,5,6,7,8
0,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall
1,Wall,→,→,→,↓,↓,Dragon,Wall,Exit
2,Wall,↑,Dragon,→,→,→,→,→,↑
3,Wall,↑,→,↑,Wall,↑,Wall,→,↓
4,→,↑,Wall,Dragon,Dragon,↑,Dragon,→,Exit
5,Wall,↑,←,→,→,→,→,→,↑
6,Wall,↑,Dragon,Wall,Wall,↑,Wall,Dragon,↓
7,Wall,→,→,→,→,↑,Dragon,Wall,Exit
8,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall


## Value Iteration

In [ ]:
def value_iteration():

    V_old = np.ones(n_states)
    V_new = np.zeros(n_states)
    iterations = 0
    while (sum(V_old == V_new) != n_states):
        V_old = np.copy(V_new)
        for s in range(n_states):
            vals = []
            for a in range(len(actions)):
                vals.append(R_s[s] + gamma * np.sum(np.multiply(trns_mx[a][s, :], V_old)))
            V_new[s] = np.max(vals)
        iterations += 1



    print("%d iterations" % iterations)

    pi_star = np.zeros(n_states)
    for s in range(n_states):
        vals = []
        for a in range(len(actions)):
            vals.append(np.sum(np.multiply(trns_mx[a][s, :], V_new)))
        pi_star[s] = np.argmax(vals)
    return pi_star, V_new

In [ ]:
pi_opt, V_opt = value_iteration()

3253 iterations


In [ ]:
# Check that V's from policy iteration/value iteration match.
assert(np.sum(np.isclose(V_star, V_opt)) == 81)
print("✓")

✓


In [ ]:
pi_opt

array([0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 2., 1., 1., 1., 1., 1., 2.,
       0., 0., 2., 0., 2., 0., 0., 0., 2., 0., 0., 2., 2., 1., 0., 2., 0.,
       2., 0., 0., 3., 2., 0., 0., 2., 0., 2., 0., 0., 3., 2., 1., 1., 2.,
       1., 1., 0., 0., 0., 2., 0., 0., 2., 0., 0., 0., 0., 0., 2., 2., 2.,
       2., 0., 0., 0., 0., 0., 1., 3., 0., 1., 3., 0., 0.])

In [ ]:

print_arr = np.flipud(np.rot90(V_opt.reshape((9, 9))))
print_list = []
for i in range(9):
    print_row = []
    for j in range(9):
        val = print_arr[i, j]
        if val == 0.0:
            print_row.append('Wall')
        elif val < 0:
            print_row.append('Dragon')
        elif val >= 99.99:
            print_row.append('Exit')
        else:
            print_row.append("%0.4f" % val)
    print_list.append(print_row)

df = pd.DataFrame(print_arr)
df

,0,1,2,3,4,5,6,7,8
0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.00,65.773,67.136,77.846,79.845,72.475,-100.000,0.000,100.000
2,0.00,55.883,-100.000,70.308,81.344,83.048,84.881,96.872,98.719
3,0.00,54.923,50.477,59.666,0.000,80.958,0.000,97.045,98.727
4,53.51,54.146,0.000,-100.000,-100.000,61.780,-100.000,88.220,100.000
5,0.00,52.504,43.936,51.091,61.007,71.786,73.947,85.185,97.573
6,0.00,43.773,-100.000,0.000,0.000,70.351,0.000,-100.000,88.406
7,0.00,47.953,48.769,58.147,59.390,60.169,-100.000,0.000,100.000
8,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [ ]:
df = pd.DataFrame(print_list)
s = df.style.applymap(color_map)
s

,0,1,2,3,4,5,6,7,8
0,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall
1,Wall,65.7731,67.1365,77.8460,79.8445,72.4751,Dragon,Wall,Exit
2,Wall,55.8829,Dragon,70.3082,81.3444,83.0485,84.8805,96.8723,98.7188
3,Wall,54.9230,50.4766,59.6664,Wall,80.9583,Wall,97.0448,98.7273
4,53.5097,54.1456,Wall,Dragon,Dragon,61.7798,Dragon,88.2204,Exit
5,Wall,52.5040,43.9360,51.0914,61.0072,71.7864,73.9466,85.1846,97.5726
6,Wall,43.7725,Dragon,Wall,Wall,70.3514,Wall,Dragon,88.4059
7,Wall,47.9530,48.7687,58.1474,59.3900,60.1689,Dragon,Wall,Exit
8,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall


In [ ]:
opt_policy_map = []
unexprbl = ['Wall', 'Dragon', 'Exit']
for i in range(9):
    row_list = []
    for j in range(9):
        val = V_opt[i * j]
        row_list.append(actions[int(pi_opt[int(i * j)])])
    opt_policy_map.append(row_list)

In [ ]:
arrow_map = np.flipud(np.rot90(np.array(opt_policy_map).reshape((9, 9))))

In [ ]:
opt_policy_map = []
arrow_map = np.flipud(np.rot90(np.array(pi_opt).reshape(9, 9)))
for i in range(9):
    row_list = []
    for j in range(9):
        val = print_list[i][j]
        if val in unexprbl:
            row_list.append(val)
        else:
            row_list.append(actions[int(arrow_map[i, j])])
    opt_policy_map.append(row_list)

In [ ]:
df = pd.DataFrame(opt_policy_map)
s = df.style.applymap(color_map)
s

,0,1,2,3,4,5,6,7,8
0,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall
1,Wall,→,→,→,↓,↓,Dragon,Wall,Exit
2,Wall,↑,Dragon,→,→,→,→,→,↑
3,Wall,↑,→,↑,Wall,↑,Wall,→,↓
4,→,↑,Wall,Dragon,Dragon,↑,Dragon,→,Exit
5,Wall,↑,←,→,→,→,→,→,↑
6,Wall,↑,Dragon,Wall,Wall,↑,Wall,Dragon,↓
7,Wall,→,→,→,→,↑,Dragon,Wall,Exit
8,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall,Wall
